In [213]:
# claiming the dataset
import pandas as pd

cxr_dataset = pd.read_csv("../features.csv")

#drop_rows = cxr_dataset.loc[cxr_dataset['label'] == 'No Finding']
#cxr_dataset.drop(drop_rows.index, inplace=True)
cxr_dataset['label'].unique()

# Getting the features needed

#get the feature columns
col_features = list(cxr_dataset.columns)
#get the label column
label = list(cxr_dataset.columns).pop()
#deleting label column from col
del col_features[0]
del col_features[len(col_features) - 1]

features = cxr_dataset.loc[:, col_features].values
labels = cxr_dataset.loc[:, label].values

cxr_dataset.head()

,Unnamed: 0,mean0,variance0,skewness0,uniformity0,snr0,mean1,variance1,skewness1,uniformity1,...,variance14,skewness14,uniformity14,snr14,mean15,variance15,skewness15,uniformity15,snr15,label
0,0,70.153934,1897.820090,0.155578,154,1.610366,67.820031,1431.488738,0.101886,149,...,0.0,0.0,0,0.0,0.000000,0.000000,0.000000,0,0.000000,COVID-19
1,1,67.943420,1640.924326,0.179606,158,1.677271,62.438356,1516.524282,0.382945,153,...,0.0,0.0,0,0.0,125.818182,59.954257,0.105233,38,16.249253,COVID-19
2,2,85.324244,1666.530997,-0.291033,153,2.090094,61.805752,1425.903805,0.326995,145,...,0.0,0.0,0,0.0,0.000000,0.000000,0.000000,0,0.000000,COVID-19
3,3,84.536424,1134.304302,-0.575772,129,2.510031,85.243692,957.789884,-0.289562,136,...,0.0,0.0,0,0.0,105.758879,247.749337,-0.316222,64,6.719092,COVID-19
4,4,97.321200,1344.070831,-0.521123,164,2.654583,102.034224,1417.579232,-0.313052,155,...,0.0,0.0,0,0.0,121.310059,518.784337,-0.373516,95,5.326027,COVID-19


In [255]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

train_features, test_features, train_lbl, test_lbl = train_test_split( features, labels, test_size=0.2, random_state=0 )

#print(test_features.shape)

scaler = StandardScaler()
pca = PCA()
lda = LinearDiscriminantAnalysis()

pipe = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('lda', lda)])

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'pca__n_components': [.95],
    'lda__solver': ['svd', 'lsqr', 'eigen'],
}
#80, .99, .98, .97, .96, .95, .94, .93, .92, .90, .89, .88, .87, .86, .85
best_model = GridSearchCV(pipe, param_grid, n_jobs=-1)

best_model.fit(train_features, train_lbl)

print("Best parameter (CV score=%0.3f):" % best_model.best_score_)
print(best_model.best_params_)

Best parameter (CV score=0.604):
{'lda__solver': 'svd', 'pca__n_components': 0.95}


In [256]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# Fitting of training dataset only

#test_features_pca = best_model.best_estimator_.steps[1][1].transform(test_features_scalar)

predictions = best_model.predict(test_features)

print("Accuracy Score -", accuracy_score(test_lbl, predictions))

print("Confusion Matrix")
print(confusion_matrix(test_lbl, predictions))

print("Classification Report")
print(classification_report(test_lbl, predictions))

Accuracy Score - 0.6833333333333333
Confusion Matrix
[[14  2  4]
 [ 1 19  0]
 [10  2  8]]
Classification Report
              precision    recall  f1-score   support

    COVID-19       0.56      0.70      0.62        20
  No Finding       0.83      0.95      0.88        20
   Pneumonia       0.67      0.40      0.50        20

    accuracy                           0.68        60
   macro avg       0.68      0.68      0.67        60
weighted avg       0.68      0.68      0.67        60

(60, 31)


In [72]:
pred_scaled = [80.69947407963937, 1269.4288426986145, -0.67499210392419, 144, 2.2649912237562297, 59.89109766637856, 1560.3788057959928, 0.4601313161779094, 142, 1.5161677386578594, 65.7536231884058, 1726.2668346985924, -0.08865097966591508, 124, 1.5825801898436467, 48.15653495440729, 1324.0773205162554, 0.4718451803368274, 134, 1.3234226306095302, 102.12631578947368, 321.77351800554015, 0.022913679792898232, 69, 5.693279637440712, 86.16784702549575, 470.625510095579, 0.186041642695024, 95, 3.9719841769645834, 0, 0, 0, 0, 0, 113.52201257861635, 119.24322613820655, -0.30065746045526565, 44, 10.395927047576272, 98.92756756756756, 338.21421300219146, 0.17355633855272548, 76, 5.379245975588224, 95.11693548387096, 258.13148738293444, 0.46243019975144345, 68, 5.920213346079804, 0, 0, 0, 0, 0, 79.04697986577182, 1382.5011485969098, -0.2991781648197267, 130, 2.1259474789571735, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 52.550424128180964, 2109.4293612715765, 0.32448564427869414, 133, 1.1441783666827834]
predictions = best_model.predict([pred_scaled])
print(predictions)
print(len(pred_scaled))

['Pneumonia']
80


In [104]:
best_est = best_model.best_estimator_.steps[1][1]
print(best_est)

PCA(n_components=0.9)
